# 48 Models

정지훈 선생의 아이디어로, 30분 단위의 데이터를 모아서 1개의 모델로 예측하는, 48개 모델의 아키텍쳐입니다.

In [1]:
import load_dtypes as ld
import warnings
warnings.filterwarnings(action='ignore')
import os, sys

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import numpy as np
from scipy.spatial import distance

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import KFold

In [2]:
TRAIN_PATH = r'C:\Users\Wyatt\wyatt37\Data\solarpanel\train\train.csv'
TEST_PATH = r'C:\Users\Wyatt\wyatt37\Data\solarpanel\test'
SUBMISSION_PATH = r'C:\Users\Wyatt\wyatt37\Data\solarpanel\sample_submission.csv'

In [205]:
train = ld.load_dtypes(TRAIN_PATH)
train.info()
submission = pd.read_csv(SUBMISSION_PATH)
test = ld.load_dtypes(TEST_PATH + '/0.csv')

C:\Users\Wyatt\wyatt37\Data\solarpanel\train\train.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52560 entries, 0 to 52559
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Day     52560 non-null  int16  
 1   Hour    52560 non-null  int8   
 2   Minute  52560 non-null  int8   
 3   DHI     52560 non-null  int16  
 4   DNI     52560 non-null  int16  
 5   WS      52560 non-null  float32
 6   RH      52560 non-null  float32
 7   T       52560 non-null  int8   
 8   TARGET  52560 non-null  float32
dtypes: float32(3), int16(3), int8(3)
memory usage: 1.1 MB
C:\Users\Wyatt\wyatt37\Data\solarpanel\test/0.csv


In [4]:
# hour와 minute을 합쳐주는 함수입니다.
def sum_hour_minute(train):
    # 0과 30으로 되어 있는 minute을 0과 0.5로 바꿉니다.
    train.Minute = round(train.Minute * 0.017, 1)
    # hour에 minute을 더해줍니다.
    train.Hour = train.Hour + train.Minute
    
    return train

In [5]:
train = sum_hour_minute(train)
train

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0.0,0.0,0,0,1.5,69.080002,-12,0.0
1,0,0.5,0.5,0,0,1.5,69.059998,-12,0.0
2,0,1.0,0.0,0,0,1.6,71.779999,-12,0.0
3,0,1.5,0.5,0,0,1.6,71.750000,-12,0.0
4,0,2.0,0.0,0,0,1.6,75.199997,-12,0.0
...,...,...,...,...,...,...,...,...,...
52555,1094,21.5,0.5,0,0,2.4,70.699997,-4,0.0
52556,1094,22.0,0.0,0,0,2.4,66.790001,-4,0.0
52557,1094,22.5,0.5,0,0,2.2,66.779999,-4,0.0
52558,1094,23.0,0.0,0,0,2.1,67.720001,-4,0.0


48개의 데이터셋으로 나눠줘야 합니다. 글로벌 변수 설정을 통해 48개의 데이터셋을 만들어줍니다. 데이터가 많지 않으니, 컬럼을 늘리지 않고 베이스라인으로만 가봅니다.

In [44]:
trains = []

for i in range(48):
    idx = i*0.5
    globals()['train_{}'.format(i)] = train[train.Hour == idx]
    globals()['train_{}'.format(i)]['TARGET_1'] = globals()['train_{}'.format(i)]['TARGET'].shift(-1)
    globals()['train_{}'.format(i)]['TARGET_2'] = globals()['train_{}'.format(i)]['TARGET'].shift(-2)
    globals()['train_{}'.format(i)] = globals()['train_{}'.format(i)].dropna()
    globals()['train_{}'.format(i)].drop(['Day', 'Minute'], axis=1, inplace=True)
    
    trains.append(globals()['train_{}'.format(i)])

1092개의 데이터를 가진 48개의 테이블을 만들었습니다.

In [45]:
trains[:2]

[       Hour  DHI  DNI   WS         RH   T  TARGET  TARGET_1  TARGET_2
 0       0.0    0    0  1.5  69.080002 -12     0.0       0.0       0.0
 48      0.0    0    0  1.6  90.660004 -10     0.0       0.0       0.0
 96      0.0    0    0  2.2  73.800003  -8     0.0       0.0       0.0
 144     0.0    0    0  1.8  77.760002 -14     0.0       0.0       0.0
 192     0.0    0    0  2.1  72.099998  -5     0.0       0.0       0.0
 ...     ...  ...  ...  ...        ...  ..     ...       ...       ...
 52224   0.0    0    0  1.0  60.779999  -4     0.0       0.0       0.0
 52272   0.0    0    0  1.4  78.230003  -6     0.0       0.0       0.0
 52320   0.0    0    0  1.8  80.360001  -8     0.0       0.0       0.0
 52368   0.0    0    0  2.6  52.590000  -1     0.0       0.0       0.0
 52416   0.0    0    0  3.4  50.470001  -1     0.0       0.0       0.0
 
 [1093 rows x 9 columns],
        Hour  DHI  DNI   WS         RH   T  TARGET  TARGET_1  TARGET_2
 1       0.5    0    0  1.5  69.059998 -12     0.

In [46]:
train_0

,Hour,DHI,DNI,WS,RH,T,TARGET,TARGET_1,TARGET_2
0,0.0,0,0,1.5,69.080002,-12,0.0,0.0,0.0
48,0.0,0,0,1.6,90.660004,-10,0.0,0.0,0.0
96,0.0,0,0,2.2,73.800003,-8,0.0,0.0,0.0
144,0.0,0,0,1.8,77.760002,-14,0.0,0.0,0.0
192,0.0,0,0,2.1,72.099998,-5,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
52224,0.0,0,0,1.0,60.779999,-4,0.0,0.0,0.0
52272,0.0,0,0,1.4,78.230003,-6,0.0,0.0,0.0
52320,0.0,0,0,1.8,80.360001,-8,0.0,0.0,0.0
52368,0.0,0,0,2.6,52.590000,-1,0.0,0.0,0.0


이제 48개의 테이블을 x_train, x_val, y_train, y_val로 찢어서 학습을 시켜야 합니다. 그리고 x_test를 받아와야 합니다.

In [47]:
# test set load

df_test = []

for i in range(81):
    file_path = TEST_PATH + '/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    # 전처리 코드 아래에 넣기---------------------------------------------------------------
    temp = sum_hour_minute(temp) # hour와 min 합치고
    df_test.append(temp[-48:]) # 뒤에서 48개, 즉 마지막 하루만 받아준다.

X_test = pd.concat(df_test)
print(X_test.shape)

(3888, 9)


In [48]:
X_test

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
288,6,0.0,0.0,0,0,0.8,80.92,-2.8,0.0
289,6,0.5,0.5,0,0,0.9,81.53,-2.9,0.0
290,6,1.0,0.0,0,0,1.0,79.91,-3.0,0.0
291,6,1.5,0.5,0,0,0.9,79.91,-3.0,0.0
292,6,2.0,0.0,0,0,0.9,77.20,-3.0,0.0
...,...,...,...,...,...,...,...,...,...
331,6,21.5,0.5,0,0,0.8,63.35,13.7,0.0
332,6,22.0,0.0,0,0,0.7,64.82,13.1,0.0
333,6,22.5,0.5,0,0,0.7,66.10,12.8,0.0
334,6,23.0,0.0,0,0,0.6,67.64,12.4,0.0


In [49]:
tests = []

for i in range(48):
    idx = i*0.5
    globals()['test_{}'.format(i)] = X_test[X_test.Hour == idx]
    #globals()['test_{}'.format(i)]['TARGET_1'] = globals()['train_{}'.format(i)]['TARGET'].shift(-1)
    #globals()['test_{}'.format(i)]['TARGET_2'] = globals()['train_{}'.format(i)]['TARGET'].shift(-2)
    #globals()['test_{}'.format(i)] = globals()['train_{}'.format(i)].dropna()
    globals()['test_{}'.format(i)].drop(['Day', 'Minute'], axis=1, inplace=True)
    
    tests.append(globals()['test_{}'.format(i)])

인덱스를 찾을 수 없네요. 인덱스는 대충 다른 변수들 짬뽕해서 찾겠습니다. 일단 81개의 테이블을 합치고, 각 시간에 대해서 찢었습니다.

In [50]:
train_0.head()

,Hour,DHI,DNI,WS,RH,T,TARGET,TARGET_1,TARGET_2
0,0.0,0,0,1.5,69.080002,-12,0.0,0.0,0.0
48,0.0,0,0,1.6,90.660004,-10,0.0,0.0,0.0
96,0.0,0,0,2.2,73.800003,-8,0.0,0.0,0.0
144,0.0,0,0,1.8,77.760002,-14,0.0,0.0,0.0
192,0.0,0,0,2.1,72.099998,-5,0.0,0.0,0.0


In [51]:
test_0.head()

,Hour,DHI,DNI,WS,RH,T,TARGET
288,0.0,0,0,0.8,80.92,-2.8,0.0
288,0.0,0,0,1.7,52.40,-10.7,0.0
288,0.0,0,0,3.3,61.40,-1.4,0.0
288,0.0,0,0,2.7,57.37,-6.7,0.0
288,0.0,0,0,2.2,67.95,-14.2,0.0


In [52]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Get the model and the predictions in (a) - (b)
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) Modeling  
    model = LGBMRegressor(objective='quantile', # quantile로 받아줘야 quatile regression이 가능합니다.
                          alpha=q, # 이건 나중에 quantile 받을 인자입니다.
                          n_estimators=10000, # 10000만번을 돌리겠다고? 미쳤어?
                          bagging_fraction=0.7,
                          learning_rate=0.027,
                          subsample=0.7)                   
                         
                         
    model.fit(X_train,
              Y_train,
              eval_metric = ['quantile'],
              eval_set=[(X_valid, Y_valid)],
              early_stopping_rounds=300, verbose=False) # 대신 얼리 스토핑을 넣었네요.

    # (b) Predictions
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles: # 9번 반복할 겁니다. quantile 마다.
        print(q)
        pred , model = LGBM(q, # 퀀틸 박아줍니다.
                            X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model) # 모델을 어펜드 해줍니다.
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1) #pred를 컨캣해줍니다.

    LGBM_actual_pred.columns=quantiles # 컬럼은 퀀틸로 박아줍니다.
    
    return LGBM_models, LGBM_actual_pred

48개의 train과 test를 넣어 각자 찢어서 학습하고, 결과를 반환하여 리스트에 저장하여 내뿜습니다.

In [65]:
# 결과 값을 받을 리스트를 만듭니다.
results_1 = []
results_2 = []

for tr, te in zip(trains, tests):
    try:
        # 단일 값이니 Hour를 지워줍니다.
        tr.drop(['Hour'], axis=1, inplace=True)
        te.drop(['Hour'], axis=1, inplace=True)
    except:
        pass
    
    # tr을 X, val 로 나눠줍니다.
    X = tr[:730]
    val = tr[730:]
    
    # train, val, x, y 로 다 찢어줍니다.
    x_train = X[X.columns[:-2]]
    y_train_1 = X['TARGET_1']
    y_train_2 = X['TARGET_2']

    x_val = val[X.columns[:-2]]
    y_val_1 = val['TARGET_1']
    y_val_2 = val['TARGET_2']
    
    _, result_1 = train_data(x_train, y_train_1, x_val, y_val_1, te)
    _, result_2 = train_data(x_train, y_train_2, x_val, y_val_2, te)
    
    results_1.append(result_1)
    results_2.append(result_2)

0.1
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
0.2
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
0.3
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
0.4
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
0.6
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
0.7
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
0.8
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
0.9
[LightGBM] [Warning] bagging

In [200]:
res

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [202]:
submission_temp = submission.copy()

In [203]:
submission_temp

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [215]:
rg = np.array((range(0, 7776, 96))) # 96개의 간격으로 벌어지는 range list를 받아줍니다.

for i, res in enumerate(results_1): # 총 48번 반복, 즉 시간별로 반복
    # submission은 0~47까지 day7, 48~95까지 day8로 되어 있습니다. 그리고 다음 테이블입니다.
    # 즉, results_1은 0, 96, 192...이렇게 넣고, 그 다음 1, 97, 193... 이렇게 넣어야 합니다.
    # 그렇게 총 81개를 넣어주면 됩니다.
    submission_temp.iloc[rg+i, 1:] = res.values

In [217]:
rg = np.array((range(0, 7776, 96))) # 96개의 간격으로 벌어지는 range list를 받아줍니다.

for i, res in enumerate(results_2): # 총 48번 반복, 즉 시간별로 반복
    # submission은 0~47까지 day7, 48~95까지 day8로 되어 있습니다. 그리고 다음 테이블입니다.
    # 즉, results_1은 0, 96, 192...이렇게 넣고, 그 다음 1, 97, 193... 이렇게 넣어야 합니다.
    # 그렇게 총 81개를 넣어주면 됩니다.
    submission_temp.iloc[rg+i+48, 1:] = res.values

In [216]:
submission_temp[16:33]

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
16,0.csv_Day7_8h00m,5.16,6.47,7.18,6.69,7.42,8.11,8.72,11.39,18.74
17,0.csv_Day7_8h30m,4.89,10.42,11.76,12.79,14.90,15.91,16.54,17.48,21.63
18,0.csv_Day7_9h00m,10.93,10.81,12.41,17.22,21.80,23.99,25.01,37.48,41.34
19,0.csv_Day7_9h30m,15.54,22.07,24.72,28.73,29.14,29.02,32.76,33.36,42.91
20,0.csv_Day7_10h00m,20.10,25.28,29.48,31.82,30.13,34.56,37.08,41.45,51.54
21,0.csv_Day7_10h30m,19.38,34.36,36.06,40.94,39.72,42.62,42.84,43.58,49.72
22,0.csv_Day7_11h00m,25.87,38.41,40.55,47.94,51.59,48.27,47.57,51.73,50.74
23,0.csv_Day7_11h30m,25.22,36.93,42.38,43.31,45.85,43.45,49.87,49.56,52.56
24,0.csv_Day7_12h00m,19.53,34.48,39.54,44.64,42.16,46.17,47.94,53.94,58.53
25,0.csv_Day7_12h30m,21.23,31.41,37.33,45.11,48.09,48.06,47.50,50.40,54.24


In [220]:
submission_temp[60:80]

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
60,0.csv_Day8_6h00m,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
61,0.csv_Day8_6h30m,0.00,0.00,0.00,0.00,0.00,0.01,0.00,-0.00,0.92
62,0.csv_Day8_7h00m,0.00,0.00,0.00,0.00,0.00,0.00,-0.11,0.01,1.18
63,0.csv_Day8_7h30m,0.75,1.22,1.54,2.28,2.44,2.54,2.76,4.03,6.71
64,0.csv_Day8_8h00m,5.74,6.78,7.12,7.64,8.76,8.89,9.99,11.71,12.18
65,0.csv_Day8_8h30m,8.86,9.69,14.07,14.85,15.72,16.82,17.27,18.83,23.75
66,0.csv_Day8_9h00m,10.90,14.17,21.19,22.39,23.37,23.60,25.31,25.88,27.06
67,0.csv_Day8_9h30m,14.79,19.55,29.58,30.94,31.22,33.13,34.13,37.73,43.07
68,0.csv_Day8_10h00m,15.64,22.17,26.66,30.84,34.33,36.07,36.40,36.07,42.46
69,0.csv_Day8_10h30m,21.05,23.46,32.42,36.04,40.97,41.08,43.98,47.90,50.25
